In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dftb_driver import dftb_plus

In [20]:
def writeXYZ(filename, names, coords):
    with open(filename, 'w') as f:
        f.write(str(len(names))+'\n')
        f.write('\n')
        for i in range(len(names)):
            f.write('{0}    {1:.8f}    {2:.8f}    {3:.8f}\n'.format(
                names[i], coords[i,0], coords[i,1], coords[i,2]))
            #f.write(str(names[i])+"  "+str(coords[i,0])+"  "+str(coords[i,1])+
            #        "  "+str(coords[i,2])+'\n')

In [3]:
f = open('modes.xyz', 'r')

In [4]:
lines = f.readlines()

In [5]:
natoms = int(lines[0].strip())
nmodes = 3 * natoms - 6
displ = np.zeros((nmodes, natoms, 3))
coords = np.zeros((natoms, 3))
names = [0] * natoms
eigen_mode = np.zeros((nmodes))

In [6]:
for i in range(natoms):
    coords[i, :] = [float(x) for x in lines[i+2].strip().split()[1:4]]
    names[i] = lines[i+2].strip().split()[0]

In [7]:
for i in range(6, nmodes): # The first six modes are neglected (translation + rotation)
    line_index = i * (natoms + 2) + 1
    eigen_mode[i-6] = float(lines[line_index].strip().split()[2])
    
    for j in range(natoms):
        line_index = i * (natoms + 2) + 2 + j
        displ[i-6, j, :] = [float(d) for d in lines[line_index].strip().split()[5:9]]

Now we test everything with the first mode only:

In [8]:
maxnorm = np.linalg.norm(displ[0, :, :], axis=1).max()

In [9]:
max_allowed_norm = 0.01 # Try many different deltaRmax
renorm = maxnorm / max_allowed_norm

In [10]:
movedCoords = np.zeros((natoms, 3, 2))
for i in range(0,2):
     movedCoords[:,:,i] = coords + (-1)**i * displ[0, :, :]/renorm

In [38]:
writeXYZ('movedcoords.xyz', names, movedCoords[:,:,0])

In [39]:
mol = dftb_plus.dftb('movedcoords.xyz')

In [40]:
Hplus = mol.H0

In [41]:
writeXYZ('movedcoords.xyz', names, movedCoords[:,:,1])

In [42]:
mol = dftb_plus.dftb('movedcoords.xyz')

In [43]:
Hminus = mol.H0

In [45]:
dHmode = (Hplus - Hminus) / (2 * max_allowed_norm)

In [46]:
dHmode

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.13831883e-03,  -2.04137072e-06,  -3.69575975e-11],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.99947208e-05,   3.41921261e-11,  -1.96359016e-06],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.62948065e-03,  -5.18424097e-06,  -9.83682832e-11],
       ..., 
       [  1.13831883e-03,  -1.99947208e-05,   2.62948065e-03, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -2.04137072e-06,   3.41921261e-11,  -5.18424097e-06, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -3.69575975e-11,  -1.96359016e-06,  -9.83682832e-11, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])